In [ ]:
import sys
sys.path.append('..')  # Add the parent folder to the sys.path
sys.path.append('../..')  # Add the parent of the parent folder to the sys.path
from pathlib import Path
from neuralhydrology.nh_run import start_run
start_run(config_file=Path("example5.yml"))

In [ ]:
run_dir = "runs/dev/run_1503_150901" #CHANGE IT

import pickle
import matplotlib.pyplot as plt
from neuralhydrology.evaluation import metrics
from math import sqrt

def mean(l):
    return sum(l)/len(l)

def stdev(l):
    m = mean(l)
    return sqrt(sum([(i-m)**2 for i in l])/len(l))

basin = '01333000'
fig, axs = plt.subplots(1,1,figsize=(14, 8)) 

for nb_epochs in range(5,26,5):
    nb_epochs = str(nb_epochs)
    nb_epochs = (3-len(nb_epochs))*'0'+nb_epochs

    with open(run_dir + "/validation/model_epoch"+nb_epochs+"/validation_results.p", "rb") as fp:
        results = pickle.load(fp)
        qsim = results[basin]['1D']['xr']['QObs(mm/d)_sim']
        qobs = results[basin]['1D']['xr']['QObs(mm/d)_obs']
        #metric_values.append(metrics.calculate_all_metrics(qobs.isel(time_step=-1),
        #                                    qsim.isel(time_step=-1)))
        if nb_epochs == '005':
            axs.plot(qobs, label='obs', alpha=1.0)
            axs.set_title("basin "+basin)
        axs.plot(qsim, label=nb_epochs, alpha=0.6)

plt.legend()
plt.show()

#for key in metric_values[0]:
#    vals = [i[key] for i in metric_values]
#    print(f"{key}: {mean(vals):.3f} ({stdev(vals):.3f})")

In [7]:
import pickle
from math import exp, tanh

class TR:

    def __init__(self, smax,s0=0) -> None:
        self.smax = smax
        self.storage = s0

    def __call__(self, p,ep):
        return self.forward(p,ep)

    def forward(self,p,ep):
        condition = 1 if p>ep else 0
        e_ir = condition * ep + (1.0 - condition) * p
        ep = ep - e_ir
        p = p - e_ir
        q = p - self.smax * (1.0 - (self.storage / self.smax)**2) * tanh(
            p / self.smax) / (1.0 + self.storage / self.smax * tanh(p / self.smax))
        e = (self.storage * (2.0 - self.storage / self.smax) *
            tanh(ep / self.smax)) / (1.0 + (1.0 - self.storage / self.smax) * tanh(ep / self.smax))
        self.storage = self.storage + p - q - e
        return q,e

class LR:
    def __init__(self,k, s0=0) -> None:
        self.k = k
        self.storage = s0

    def __call__(self, p):
        return self.forward(p)

    def forward(self, x_in):
        old_storage = self.storage
        self.storage=x_in/self.k+(self.storage-x_in/self.k)*exp(-self.k)
        q=x_in-(self.storage-old_storage)
        return q


def get_area(basin):
    with open("../../data/fake/basin_mean_forcing/daymet/02/" + basin + "_lump_cida_forcing_leap.txt",
              'r') as fp:
        # load area from header
        fp.readline()
        fp.readline()
        area = int(fp.readline())
    return area


def from_mm_d_to_mcube_feet(basin, x):
    area = get_area(basin)
    x = x * (area * 10**6) / (28316846.592*86400)
    return x

def generate_data(basin,smax,k1,k2,p1):
    pkl_path = '../../data/pet.pkl'
    with open(pkl_path, 'rb') as f:
        pets = pickle.load(f)[basin]['PET(mm/d)'].tolist()

    file_path = "../../data/CAMELS_US/basin_mean_forcing/daymet/02/"+basin+"_lump_cida_forcing_leap.txt"
    precip = []
    dates = []
    with open(file_path) as f:
        for _ in range(9410):
            l = f.readline()
        pets = pets[9410-5:9410-5 + 1094]
        for _ in range(1095):
            l = l.split('\t')
            precip.append(float(l[2]))
            dates.append(l[0])
            l = f.readline()
        print(pets[0],pets[-1], precip[0], precip[-1])
    tr = TR(smax)
    lr1 = LR(k1)
    lr2 = LR(k2)
    with open("../../fake_"+basin+"_2.txt",'w') as f:
        for d,p,e in zip(dates, precip,pets):
            output_tr = tr(p,e)[0]
            output = lr1((p1) * output_tr) + lr2((1-p1) * output_tr)
            #output = from_mm_d_to_mcube_feet(basin, output)
            f.write(' '.join([basin, d[:-3], str(output)]))
            f.write('\n')


generate_data('02027500', 320.167324039440, 0.06789262074757384, 3.18088315483156, 1-0.113816928208218)

In [ ]:
nb_archs = 5
ms = []
stds = []
for i in range(1,nb_archs+1):
    ms.append([])
    stds.append([])
    run_dir = "runs/dev/arch"+str(i)
    metric_values = []
    with open(run_dir + "/validation/model_epoch"+nb_epochs+"/validation_results.p", "rb") as fp:
        results = pickle.load(fp)
        for i,basin in enumerate(basins):
            qsim = results[basin]['1D']['xr']['QObs(mm/d)_sim']
            qobs = results[basin]['1D']['xr']['QObs(mm/d)_obs']
            metric_values.append(metrics.calculate_all_metrics(qobs.isel(time_step=-1),
                                                qsim.isel(time_step=-1)))

    for key in metric_values[0]:
        vals = [i[key] for i in metric_values]
        ms[-1].append(round(mean(vals),3))
        stds[-1].append(round(stdev(vals),3))

res = "\\textbf{}" + "".join([" & \\textbf{Architecture "+str(i)+"}" for i in range(1,nb_archs+1)]) + " \\\\ \hline \n"
for i,k in enumerate(metric_values[0].keys()):
    res += k
    for a in range(len(ms)):
        res += ' & '+str(ms[a][i]) #+ '(' + str(stds[a][i])+')'
    res += "\\\\ \hline \n"

print(res)